In [7]:
import pandas as pd
import numpy as np
import ipywidgets
from ipywidgets import widgets
from ipywidgets import widgets,interactive,interact, HBox, Layout,VBox, Box
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [13]:
#### load Data 
WRUP = pd.read_csv('WRUP2.csv',index_col=0)
RecRt = pd.read_csv('RecRate2.csv',index_col=0)
HSLTypes = pd.read_csv('AppHSLdict.csv', index_col=0)

global BNNDict, IMDict
BNNDict = {'Recession Rate':RecRt, 'Wave Runup':WRUP, 'HSLTypes' :HSLTypes}
IMDict = {'Recession Rate':'NWPlot_RRWI2.png', 'Wave Runup':'NWPlot_WRUP2.png'}

## spatial Analysis ##
DFAll = pd.read_csv('APPDATA.csv')

TabColumns = ['Transect_Name', 'BarCount', 'AveWavelength', 'NearshoreSlope', 'CurveRatio', 'BarDistance', 
              'BarDepth', 'Shoreline_Type', 'BarPresence', 'WaveIndx', 'Enrg','EnrgClass','RecRate','RecClass']

Colors = ['NearshoreSlope','BarCount', 'AveWavelength', 'BarDistance', 'BarDepth', 
          'CurveRatio', 'Enrg','RecRate','EnrgClass','RecClass']
Sizes = ['BarDistance','BarCount', 'AveWavelength','NearshoreSlope',  'BarDepth', 
         'CurveRatio', 'Enrg','RecRate','EnrgClass','RecClass']

RoundDict = {'AveWavelength':0, 'NearshoreSlope': 4,'BarDistance':0, 
             'BarDepth':2, 'RecRate':2, 'CurveRatio':4, 'WaveIndx':3, 'Enrg':3}

for col in RoundDict.keys():
    DFAll[col] = DFAll[col].round(RoundDict[col])
    
def ScaleVal(Vals):
    S = Vals - Vals.quantile(0.10)
    S[S<0] = 0 
    S = (S/S.quantile(0.90))
    S[S>1] = 1
    S *= 20
    S[S<1] = 1
    return S.round(0)

In [9]:
# def BBNPlotlyGraph(Variable, BayesModel): 
def BBNPlotlyGraph(Variable,BayesModel):
    BayesModel = BayesModel
    Var1 = Variable
    DF = BNNDict[BayesModel].copy()
    Vars = ['BarCount','SLType','AveWavelength','BarDistance','BarDepth','NearshoreSlope','BW','CurveRatio','WaveIndx','RecRate','Enrg']
    CompVars = [x for x in Vars if x in DF.columns and x not in [Var1,'SLType']]

    YRows = int(round((len(CompVars)/3)+0.5))
    XCols = 3

    count = 0
    fig = make_subplots(rows=YRows, cols=XCols,horizontal_spacing=0.06,vertical_spacing=0.09)
    for xs in range(YRows):
        for ys in range(XCols):
            if count < len(CompVars):
                Data = pd.pivot_table(DF[DF[CompVars[count]] != 'nan'],values= 'Class', index = Var1, columns = CompVars[count],   aggfunc = 'mean')
                # count = pd.pivot_table(DF,values= 'FltMap', index = Var1, columns = Var2,   aggfunc = 'count')
                Data = Data[Data.columns.astype(float).sort_values()]
                Data.columns = Data.columns.astype(str)
                Data = Data.loc[Data.index.astype(float).sort_values(),:]
                Data.index =Data.index.astype(str)
                Data['Total'] = Data.mean(axis=1)
                Data.loc['Total',:] = Data.mean()
                Data.loc['Total','Total'] = np.nan

                x = Data.columns.astype(str).values
                y = Data.index.astype(str).values

                fig.add_trace(go.Heatmap(x =[i[:6] for i in x],y=[i[:6] for i in y[::-1]], 
                                         z = Data.values[::-1,:], coloraxis = "coloraxis",zauto=False), row = xs+1,col = ys+1)
                fig.update_xaxes(title_text=CompVars[count],title_standoff = 0,row = xs+1, col = ys+1)
                fig.update_xaxes(tickangle=-25)

                count+=1

    for i in range(1,YRows+1):
        fig.update_yaxes(title_text=Var1,tickangle=-45,title_standoff = 0, row=i, col=1)

    fig.update_layout(coloraxis = {'colorscale':'ylgnbu_r'})
    fig.update_layout({"coloraxis_cmin": 0.5,"coloraxis_cmax": 1.5,})
    fig.update_xaxes(side="top")
    fig.update_coloraxes(showscale=False)
    fig.update_layout(width=700,height=230*YRows,font=dict(size=10),margin=dict(l=10, r=10, t=10, b=10))

    fig.show()
    
def SLPlotlyGraph(SLComp): 
    Var1 = 'SLType'
    Var = SLComp
    DF = BNNDict['Recession Rate'].copy()
    HSLTypes = BNNDict['HSLTypes'].copy()
    Data = pd.pivot_table(DF[DF[Var1] != 'nan'],values= 'Class', index = Var, columns = 'SLType',   aggfunc = 'mean')

    Data = Data[Data.columns.astype(float).sort_values()]
    Data.columns = HSLTypes.loc[Data.columns,'Shoreline_Type_Description'].values
    Data = Data.loc[Data.index.astype(float).sort_values(),:]
    Data.index = Data.index.astype(str)
    Data['Total'] = Data.mean(axis=1)
    Data.loc['Total',:] = Data.mean()
    Data.loc['Total','Total'] = np.nan

    # return Data.index
    # x =   HSLTypes.loc[Data.columns.astype(float).astype(int),'Shoreline_Type_Description'].values
    x = Data.columns.astype(str).values
    y = Data.index.astype(str).values

    fig = make_subplots(rows=1, cols=1,horizontal_spacing=0.03,vertical_spacing=0.09)
    fig.add_trace(go.Heatmap(x = x,y=[i[:6] for i in y[::-1]], 
                             z = Data.values[::-1,:], coloraxis = "coloraxis",zauto=False))
    fig.update_xaxes(title_text='Shoreline Type Analysis',title_standoff = 0)
    # fig.update_yaxes(tickangle=-90)

    fig.update_layout(coloraxis = {'colorscale':'ylgnbu_r'})
    fig.update_layout({"coloraxis_cmin": 0.5,"coloraxis_cmax": 1.5,})
    fig.update_xaxes(side="bottom")
    fig.update_layout(width=1400,height=300,font=dict(size=11),margin=dict(l=20, r=20, t=20, b=20))
    fig.update_yaxes(title_text=Var,tickangle=-45)

    fig.show()
    
spcval = 0.25
def PredPlot(Data,Title):
    SCTS = []
    lens = Data.shape[1]
    for i in range(1,4):
        SCTS.append(go.Scatter(y=Data[i], x=np.linspace((i-spcval),(i+spcval),lens), mode='markers', name='',
                       marker = dict(size=8, color=PColors[:lens]),hovertemplate ='<b>%{text}</b>',
                       text = Data[0]))

    fig = go.FigureWidget(SCTS)
    fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'},
                      height=500,showlegend=False,
                      xaxis = dict(tickmode = 'array',tickvals = [1, 2, 3],ticktext = Predcats),
                     margin={"r":50,"t":10,"l":0,"b":10})
    fig.update_yaxes(gridcolor = "#BCCCDC",showgrid=True)
    fig.update_xaxes(gridcolor = "#BCCCDC",showgrid=True)
    fig.update_yaxes(title_text = "Prediction Likelihood",title_standoff = 5,range=[0, 1])
    fig.update_xaxes(side='top',title_text = Title,title_standoff = 5, range=[.2, 3.7])
    return fig



def BOXPLOTS(BOXFRAME):
    BOXPLOT = make_subplots(rows=1, cols=len(BoxCols),horizontal_spacing=0.02,vertical_spacing=0)
    
    for i in range(len(BoxCols)):
        BOXPLOT.add_trace(go.Box(y=BOXFRAME[i],name=BoxCols[i],whiskerwidth=0.2,boxmean=True,boxpoints=False ), row = 1,col = i+1)
        
    Random = BOXFRAME[:,[0]]
    for i in range(len(BoxCols)):    
        BOXPLOT.add_trace(go.Box(y=Random[i],name=BoxCols[i],whiskerwidth=0.2,boxpoints='all', jitter=0.2, pointpos=-1.5,marker_color = 'rgba(0, 0, 0,.5)',
                             marker = dict(size=8), line = dict(color = 'rgba(0,0,0,0)'),fillcolor = 'rgba(0,0,0,0)'), 
                      row = 1,col = i+1)
        BOXPLOT.update_yaxes(gridcolor = "#BCCCDC",showgrid=True,row = 1, col = i+1)
    BOXPLOT.update_xaxes(side="top")
    BOXPLOT.update_yaxes(tickangle=-35)
    BOXPLOT.update_layout(margin={"r":80,"t":10,"l":10,"b":10})
    BOXPLOT.update_layout(height = 400, showlegend=False)
    BOXPLOT.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'},hovermode=False)
    BP = go.FigureWidget(BOXPLOT)
    
    return BP

In [10]:
HoverTEXT = '''<b>%{hovertext}</b><br><br>BarDistance=%{customdata[0]}<br>BarCount=%{customdata[1]}
                <br>AveWavelength=%{customdata[2]}<br>NearshoreSlope=%{marker.color}<br>BarDepth=%{customdata[4]}
                <br>CurveRatio=%{customdata[5]}<br>Enrg=%{customdata[6]}<br>RecRate=%{customdata[7]}
                <br>EnrgClass=%{customdata[8]}<br>RecClass=%{customdata[9]}<extra></extra>'''

In [11]:
BoxCols = ['BarCount', 'AveWavelength', 'BarDistance', 'BarDepth', 'CurveRatio','NearshoreSlope','WaveIndx', 'Enrg', 'RecRate']
TABFRAME = DFAll[TabColumns].values.T
BOXFRAME = DFAll[BoxCols].values.T

ENRGDF = DFAll[['Transect_Name','Enrg_1','Enrg_2','Enrg_3']].copy().values.T
RECDF = DFAll[['Transect_Name','RecRate_1','RecRate_2','RecRate_3']].copy().values.T

DFHold = DFAll.fillna(DFAll.min()).reset_index(drop=True).copy()
DFHold.loc[~DFHold.RecTrain,'RecClass'],DFHold.loc[~DFHold.EnrgTrain,'EnrgClass'] = -1, -1

for s in Sizes:
    DFHold[s +'_S'] = ScaleVal(DFHold[s])
    
Predcats = ['Low','Med','High']
PColors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', '#008080', '#e6beff', '#9a6324', 
    '#fffac8', '#800000', '#aaffc3', '#808000', '#ffd8b1', '#000075', '#808080', '#ffffff', '#000000']

In [12]:
BNNBut = ipywidgets.Button(description='BBN Analysis')
SpatialBut = ipywidgets.Button(description='Spatial Analysis')
box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='center',width='50%')

out = ipywidgets.Output(layout=ipywidgets.Layout(border ='1px solid black',width='100%'))

BNNRec = widgets.Image(value=open(IMDict['Recession Rate'], 'rb').read(), format='png', width=600, height=600)
BNNWave = widgets.Image(value=open(IMDict['Wave Runup'], 'rb').read(), format='png', width=600, height=600)
BNNNode1 = HBox([BNNRec],layout=box_layout)
BNNNode2 = HBox([BNNWave],layout=box_layout)
BBNTop = HBox([BNNNode1,BNNNode2])

def BBNPRec(CompVar):
    BBNPlotlyGraph(CompVar,BayesModel = 'Recession Rate')
def BBNPWave(CompVar):
    BBNPlotlyGraph(CompVar,BayesModel = 'Wave Runup')

BNN1 = interactive(BBNPRec,
                 CompVar = ['BarCount','AveWavelength','BarDistance','BarDepth','NearshoreSlope','BW','CurveRatio','WaveIndx','RecRate'])
BNN2 = interactive(BBNPWave,
                 CompVar = ['BarCount','AveWavelength','BarDistance','BarDepth','NearshoreSlope','BW','CurveRatio','Enrg'])
SLType = interactive(SLPlotlyGraph,
                 SLComp = ['BarCount','AveWavelength','BarDistance','BarDepth','NearshoreSlope','BW','CurveRatio','WaveIndx','RecRate'], )

BNN1Box = HBox([BNN1],layout=box_layout)
BNN2Box = HBox([BNN2],layout=box_layout)

BBNBoc1 = HBox([BNN1Box,BNN2Box])

def BNNButClick(click):
    with out:
        out.clear_output() 
        display(BBNTop)
        display(VBox([BBNBoc1,SLType]))
        
        
######## SPATIAL ANALYSIS SECTION #########     
    
fig = px.scatter_mapbox(DFHold, lat="lat", lon="lon", color="NearshoreSlope", size='BarDistance_S',size_max=20, zoom=6, height=500,
                        hover_name="Transect_Name", hover_data=Sizes)
f = go.FigureWidget(fig)
f.update_layout({"coloraxis_cmin": DFHold['NearshoreSlope'].quantile(.1),
                "coloraxis_cmax": DFHold['NearshoreSlope'].quantile(.9)},
                coloraxis_colorbar=dict(title=None))

f.data[0]['hovertemplate'] = HoverTEXT
scatter = f.data[0]
scatter.marker.opacity = 0.5

#create reanimation script to recolor and size elements
def update_axes(Color, Size):
    with f.batch_update():
        f.data[0].marker.color = DFHold[Color]
        f.update_layout({"coloraxis_cmin": DFHold[Color].quantile(.1),
                         "coloraxis_cmax": DFHold[Color].quantile(.9),})
        f.data[0].marker.size = ScaleVal(DFHold[Size+'_S'])
        f.data[0]['hovertemplate'] = HoverTEXT
        
axis_dropdowns = interactive(update_axes, Color = Colors, Size = Sizes)
f.update_layout({'mapbox': {'style': 'stamen-terrain'}},margin={"r":170,"t":0,"l":10,"b":0})

        
## Create a table FigureWidget that updates on selection from points in the scatter plot of f
t = go.FigureWidget([go.Table(
    header=dict(values=TabColumns, fill = dict(color='#C2D4FF'), align = ['left'] * 5),
    cells =dict(values=TABFRAME, fill = dict(color='#F5F8FF'), align = ['left'] * 5))])
t.update_layout(height=300,margin={"r":220,"t":0,"l":10,"b":10})

## boxplots of data
BP = BOXPLOTS(BOXFRAME)

## Predictions
ENRGFIG = PredPlot(ENRGDF[:,[1]],"WvEnrg Reduction Est.")
RECFIG  = PredPlot(RECDF[:,[1]],"Rec. Rate Est.")

def selection_fn(trace,points,selector):
    t.data[0].cells.values = TABFRAME[:,points.point_inds].tolist()
    
    with BP.batch_update():
        for i in range(len(BoxCols)):
            BP.data[i+len(BoxCols)]['y'] = BOXFRAME[i,points.point_inds]
    
    Data = ENRGDF[:,points.point_inds]
    lens = Data.shape[1]
    with ENRGFIG.batch_update():
        for i in range(1,4):
            ENRGFIG.data[i-1]['marker']['color'] = (PColors*(lens//20+1))[:lens]
            ENRGFIG.data[i-1]['text'] = Data[0]
            ENRGFIG.data[i-1]['x'] = np.linspace((i-spcval),(i+spcval),lens)
            ENRGFIG.data[i-1]['y'] = Data[i]
            
    Data = RECDF[:,points.point_inds]
    lens = Data.shape[1]
    with RECFIG.batch_update():
        for i in range(1,4):
            RECFIG.data[i-1]['marker']['color'] = (PColors*(lens//20+1))[:lens]
            RECFIG.data[i-1]['text'] = Data[0]
            RECFIG.data[i-1]['x'] = np.linspace((i-spcval),(i+spcval),lens)
            RECFIG.data[i-1]['y'] = Data[i]

scatter.on_selection(selection_fn)
# def selection_fn(trace,points,selector):

TBLBOX_Layout = Layout(display='flex',flex_flow='column',width='95%')
BPbx = HBox([BP],layout=TBLBOX_Layout)
tbx = HBox([t],layout=TBLBOX_Layout)

Map_Layout = Layout(display='flex',flex_flow='column',width='55%')
MAPbx = HBox([f],layout=Map_Layout)

PRED_Layout = Layout(display='flex',flex_flow='column',width='20%')
ENRGbx = HBox([ENRGFIG],layout=PRED_Layout)
RECbx = HBox([RECFIG],layout=PRED_Layout)   

def SpatialButClick(click):
    with out:
        out.clear_output() 
        
        display(VBox([HBox(axis_dropdowns.children),HBox([MAPbx,ENRGbx,RECbx]),BPbx,tbx]))

        
BNNBut.on_click(BNNButClick)
SpatialBut.on_click(SpatialButClick)


display(HBox([BNNBut,SpatialBut]))
display(out)
display("*** All data is preliminary")

C:\Anaconda3\envs\NSGeoIndex_app\lib\site-packages\jupyter_client\session.py:716: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



Output(layout=Layout(border='1px solid black', width='100%'))

'*** All data is preliminary'